In [2]:
import pandas as pd  # грузим библиотеки
import json
import requests
from tqdm import tqdm
import time
import gspread
from df2gspread import df2gspread as d2g
from oauth2client.service_account import ServiceAccountCredentials
import sys

job_title = "'Аналитик данных' and 'data analyst'"  # пишем запрос
area = 1  # зона поиска (1 - Москва)


def getPage(page=0):  # пишем функцию запроса

    params = {
        'text': job_title,
        'area': area,
        'page': page,
        'per_page': 100
    }

    # отправляем запрос на сервер
    req = requests.get('https://api.hh.ru/vacancies', params)
    data = req.content.decode()  # присваеваем и декодируем ответ
    req.close()
    return data


data = []
# итерируемся,tqdm для визуалиции прогресс-бара
for page in tqdm(range(0, 20), desc='Парсим по запросу '+job_title):
    jsObj = json.loads(getPage(page))  # загружаем страницу
    data.append(jsObj)  # добавляем в список


def highlighttext(column):
    # фунция для отчистки от лишних символов
    return column.str.replace('[<\>\/]', '', regex=True).str.replace('highlighttext', '', regex=True)


df = pd.json_normalize(data, record_path=['items']).rename(
    columns=lambda c: c.replace('.', '_'))  # заменим . на _ в названии колонок


df = df[['id', 'name',#соберем датафрейм
         'published_at',
         'url',
         'area_name',
         'salary_from',
         'salary_to',
         'salary_currency',
         'employer_name',
         'employer_url',
         'snippet_requirement',
         'snippet_responsibility',
         'schedule_name', 'address_raw', 'address_lat',
         'address_lng', 'address_metro_station_name']]

df.snippet_responsibility = highlighttext(df.snippet_responsibility)#применяем функцию
df.snippet_requirement = highlighttext(df.snippet_requirement)


timestr = time.strftime("%Y%m%d-%H%M%S")
csv_name = job_title+".csv"#вариант без времени в названии
csv_name_with_timestamp = job_title+timestr+".csv"#вариант с временем в названии
df.to_csv(csv_name, index=False)  # сохраняем
print('Парсинг выполнен'+' в', pd.Timestamp.now())

"""
# Выгрузка в Google Sheets
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

my_mail = 'your_mail'#почта
path_to_credentials = 'crdentials.json'

# Authorization
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'your_keyfile_name.json', scope)
gs = gspread.authorize(credentials)


table_name = 'HH'
sheet = gs.create('HH')

sheet.share(my_mail, perm_type='user', role='writer')

spreadsheet_name = 'HH'
sheet = 'HH'
d2g.upload(df, table_name, sheet, credentials=credentials, row_names=True)
print('Сохранено как '+sheet+' в', pd.Timestamp.now())
"""


Парсим по запросу 'Аналитик данных' and 'data analyst': 100%|██████████| 20/20 [00:20<00:00,  1.05s/it]

Парсинг выполнен в 2022-07-05 13:52:07.734887


"\n# Google\nscope = ['https://spreadsheets.google.com/feeds',\n         'https://www.googleapis.com/auth/drive']\n\nmy_mail = 'your_mail'\npath_to_credentials = 'crdentials.json'\n\n# Authorization\ncredentials = ServiceAccountCredentials.from_json_keyfile_name(\n    'keyfile_name.json', scope)\ngs = gspread.authorize(credentials)\n\n\ntable_name = 'HH'\nsheet = gs.create('HH')\n\nsheet.share(my_mail, perm_type='user', role='writer')\n\nspreadsheet_name = 'HH'\nsheet = 'HH'\nd2g.upload(df, table_name, sheet, credentials=credentials, row_names=True)\nprint('Сохранено как '+sheet+' в', pd.Timestamp.now())\n"

In [5]:
df.head(2)


,id,name,published_at,url,area_name,salary_from,salary_to,salary_currency,employer_name,employer_url,snippet_requirement,snippet_responsibility,schedule_name,address_raw,address_lat,address_lng,address_metro_station_name
0,67439152,Data expert / Главный эксперт по анализу данных,2022-07-04T09:26:13+0300,https://api.hh.ru/vacancies/67439152?host=hh.ru,Москва,250000.0,500000.0,RUR,Double Data,https://api.hh.ru/employers/1300820,Вы имеете релевантный опыт сбора иили анализа ...,Сбором данных из социальных сетей и прочих отк...,Удаленная работа,NaN,NaN,NaN,NaN
1,50932609,Data analyst / Аналитик данных,2022-07-04T14:20:43+0300,https://api.hh.ru/vacancies/50932609?host=hh.ru,Москва,200000.0,350000.0,RUR,24Н Софт,https://api.hh.ru/employers/2515455,Знание и отличное владение SQL обязательно. От...,"Анализ данных (DWH, BigQuery) по различным асп...",Гибкий график,"Москва, Шипиловская улица, 28А",55.618745,37.715138,Домодедовская
